In [1]:
from importlib import reload
from bs4 import BeautifulSoup
import requests as requests
import time
import re
import pandas as pd
import numpy as np
from scripts import user
from user_modules import scraping_item as si


## Test chunck

In [36]:
titles = []

start = time.time()

for i in range(1,5):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue
    
end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")
print(titles)

The time of execution of above program is : 4.7752416133880615 s
['El Salvador Makes Bitcoin Legal Tender', 'stacker news soft launch AMA']


## Collect post header with all the related data

In [ ]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

banner_data = []

for i in banner.find_all('span'):
    banner_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print([titolo, banner_data])


# Scraping Items of stacker.news

In [33]:
## ALBERTO
df = pd.DataFrame(columns=['titolo', 'numero commenti', 'boost', 'sats', 'betha',"commentors"])
n_posts = range(1,20)

start = time.time()

for number in n_posts:
    # print("Doing", number)
    url_posts = f'https://stacker.news/items/{number}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        contenuto=soup.find('div', class_="text_text__TTY3L").get_text()
    except:
        continue
    try:
        titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
    except:
        continue
    try:
        banner = soup.find('div', class_='item_other__MjgP3')
    except:
        continue
    try:
        commento = soup.find('a', class_='text-reset position-relative').get_text()
    except:        
        continue
    try:
        a_elements = soup.find_all('a')
    except:
        continue
    
    at_elements = []

    for el in a_elements:
        links = el.get_text()
    
        if links.startswith('@'):
            at_elements.append(el)
    
    commentors_list=[]
    for ind in range(0,len(at_elements)):
        commentors_list.append(at_elements[ind]["href"])


    banner_data = []
    for i in banner.find_all('span'):
        banner_data.append(i.text)
    index=number-1
    for b in banner_data:
        if "boost" in b:
            df.at[index, 'boost'] = b
        if "sats" in b:
            df.at[index, 'sats'] = b
        if "@" in b:
            df.at[index, 'betha'] = b   # TODO: insert the datetime extraction function

    df.at[index, 'titolo'] = titolo
    df.at[index, 'numero commenti'] = commento
    df.at[index, 'contenuto'] = contenuto
    df.at[index, 'commentors'] = commentors_list

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

df


/tmp/ipykernel_11228/4172341155.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '100k posts in 18 months is pretty damn good
https://stacker.news/items/100000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'contenuto'] = contenuto


The time of execution of above program is : 18.337690114974976 s


,titolo,numero commenti,boost,sats,betha,commentors,contenuto
0,El Salvador Makes Bitcoin Legal Tender,13 comments,4 boost,603 sats,@k00b 11 Jun 2021,"[/k00b, /ln123, /k00b, /k00b, /OneOneSeven, /e...",100k posts in 18 months is pretty damn good\nh...
1,stacker news soft launch AMA,30 comments,NaN,2 sats,@k00b 11 Jun 2021,"[/k00b, /sha256, /k00b, /banana, /utxoclub, /k...",Figured I'd make a place to raise questions ab...
7,The Lightning Conference 2019: Alex Bosworth,2 comments,NaN,0 sats,@k00b 11 Jun 2021,"[/k00b, /k00b, /k00b, /k00b, /kr, /ekzyis]",I'm trying to add liquidity to the stacker new...


## Reframing the previous code
Main steps in the code
1. Retrieve item webpage provided the item code
2. Detect item type:
    - Comment or post ?
    - If post, which kind of post:
        1. Discussion
        2. Link
        3. Poll
        4. Bounty
        5. Job
3. Retrieve title
4. Retrieve banner
    - Extract number of comment, **compulsory**
    - Extract stacked amount by the item, **if present**
    - Extract Boost value, **if present**
    - Extract username, **compulsory**
    - Extract timestamp, **compulsory**
    - Extract badge, **compulsory**
5. Extract amount stacked by comments, **compulsory**
6. Extract item code of comments **OR** extract user that commented

**Note that**:
- Some items do not have the stacked amount nor the possibility to receive sats. For example the user @saloon created all this kind of posts. Is he/she a bot? Is it an 'official bot' of the forum and so it's not possible to give sats to it?


### Detecting item types
First of all we need to determine the kind of item we are working with.
The typology is described by the tag of the post in the post-creation.

In [32]:
reload(si)

NA = None




True

# Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

## Modularize the code - user profile scraping
Define functions to modularize and simplify the user profile scraping.
The functions are defined in `user_modules/scraping_user.py` and have been tested with `tests/test_scraping_user.py`. To run the tests open a the terminal/CMD and run the script `tests/test_scraping_user.py` from there, running it from a JupyterNotebook could raise errors.
The testing script tested every function in some corner cases (missing data/request error). 

**From the following two approaches we can create the final script for scraping the user profiles**.
The next step would be the creation of a function that loops through the user list and assigns to the rows in the dataframe the values returned from the functions defined in `user_modules/scraping_user.py`.

In [142]:
# TODO: integrate this method into the general scraping script for profile scraping

user_list = np.array(['Monotone',
                      'TNStacker',
                      'kale',
                      'DiracDelta',
                      'kr',
                      'moscowTimeBot',
                      'mpuels',
                      'blockstream_official',
                      'nym',
                      'random_'
                      ])
                      
start = time.time()

for i in np.nditer(user_list):
    print(su.get_total_stacked(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")

123353
53789
357270
181422
1114441
6427
0
50703
164002
1369008
The average time of execution of the above program for one user is : 0.6400596380233765 s


In [145]:
user_list = ['Monotone',
             'TNStacker',
             'kale',
             'DiracDelta',
             'kr',
             'moscowTimeBot',
             'mpuels',
             'blockstream_official',
             'nym',
             'random_',
             ]
                      
start = time.time()

for i in user_list:
    print(su.get_total_stacked(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")

123353
53789
357270
181422
1114441
6427
0
50703
164002
1369008
The average time of execution of the above program for one user is : 0.691546106338501 s
